In [6]:
# Import required modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os.path
import pickle

import argparse
import torch
import torch.nn as nn

import random
import numpy.random
import torch.random

from datetime import datetime

import warnings
warnings.filterwarnings(action='ignore')

In [8]:
# Define variables
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

# 2023-05-02
# For Using GPU (CUDA)
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2023-05-02
# For using tensorboard
args.istensorboard = False

args.total_episode = 100
args.learning_rate = 0.1 # Learning rate Alpha
args.boltzmann_tau_initial = 5 # Initial temperature parameter at Boltzmann policy, Tau
args.total_reward = 0
args.epsilon = 0.1

args.max_iteration = 50 # Maximum iteration num. of algorithm, MAX_STEPS
args.replay_batch = 100 # Replay batch size, B
args.nn_update_num = 20 # CNN update number, U: [(1) Constant num. of iteration], (2) Lower limit of loss function value
args.batch_size = 20 # Batch size, N
args.replay_memory = [] # (1) Using list class, (2) Replay memory class by SLM Lab (page. 105~108)

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120 # ft
args.gridsize_y = 120 # ft

args.time_step = 120 # days
args.total_production_time = 600 # days

# 2023-05-02: For reproduction
args.random_seed = 202022673
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# State: Pressure distribution, Oil saturation

# Action: Well placement (Coordinate of well location)

# Environment: Reservoir simulator

# Reward: NPV at each time segment

args.discount_rate = 0.1 # Used for calculation of NPV
args.discount_factor = 1 # Used for Q-value update

# Directory-related arguments
args.filepath = 'data'
args.simulation_directory = 'simulation'
args.save_directory = 'variables'
args.ecl_filename = '2D_JY_Eclrun'
args.frs_filename = '2D_JY_Frsrun'
args.perm_filename = '2D_PERMX'
args.position_filename = '2D_POSITION'
args.constraint_filename = '2D_CONSTRAINT'

# Dynamic data for State
args.input_flag = ('PRESSURE', 'SOIL')

In [3]:
# Define Functions (or Methods) and CNN Structure

################################# Reading Eclipse Dynamic Data #################################
# idx: simulation file index
# tstep_idx: simulation time step index
# filename: simulation file name
# dynamic_type: dynamic data type to collect ('PRESSURE' or 'SOIL')
def _read_ecl_prt(args, idx: int, tstep_idx: int, filename: str, dynamic_type: str) -> list:
    # Check if dynamic type input is (1) 'PRESSURE', (2) 'SOIL'
    if not dynamic_type in ['PRESSURE', 'SOIL']:
        print("Assign correct dynamic data output type!: 'PRESSURE', 'SOIL'")
        return -1

    # File IO
    # 1. Open .PRT file
    with open(args.simulation_directory+'\\'+filename+'_'+str(idx)+'.PRT') as file_read:
        line = file_read.readline()
        if dynamic_type == 'PRESSURE':
            # 2. Fine the location of dynamic data (PRESSURE case)
            while not line.startswith(f"  {dynamic_type} AT   {args.time_step * tstep_idx}"):
                line = file_read.readline()
            # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
            for i in range(1,10+1):
                line = file_read.readline()
            # 4. Collect dynamic data
            lines_converted = []
            for i in range(1, args.gridnum_y+1):
                lines_converted.append([element.strip() for element in line.split()][3::])
                line = file_read.readline()
        elif dynamic_type == 'SOIL':
            # 2. Fine the location of dynamic data (SOIL case)
            while not line.startswith(f"  {dynamic_type}     AT   {args.time_step * tstep_idx}"):
                line = file_read.readline()
            # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
            for i in range(1,10+1):
                line = file_read.readline()
            # 4. Collect dynamic data
            lines_converted = []
            for i in range(1, args.gridnum_y+1):
                lines_converted.append([element.strip() for element in line.split()][3::])
                line = file_read.readline()

    return lines_converted
##########################################################################################

################################# Running ECL Simulator ##################################
# program: 'eclipse' or 'frontsim'
# filename: simulation file name (ex. 2D_JY_ECLRUN_1)
def _run_program(args, program: str, filename: str):
    # Check if dynamic type input is (1) 'eclipse', (2) 'frontsim'
    if not program in ['eclipse', 'frontsim']:
        print("Use correct simulator exe file name!: 'eclipse', 'frontsim'")
        return -1
    command = fr"C:\\ecl\\2009.1\\bin\\pc\\{program}.exe {filename} > NUL"
    os.chdir(args.simulation_directory)
    os.system(command)
    os.chdir('../')
##########################################################################################

###################################### CNN Structure #####################################
class CNN(nn.Module):
    # "Deep Reinforcement Learning for Generalizable Field Development Optimization", Jincong He et al, 2021
    # Convolution-ReLU-Residual block (==Original-Convolution-ReLU-Convolution+Original-ReLU)
    # argument로 args를 받아와, input_flag가 설정되어 있으면 그 수에 맞게 채널 수를 조정해주고, 없으면 기본값인 3으로 설정됨
    def __init__(self, args):
        super().__init__()
        if args.input_flag: self.num_of_channels = len(args.input_flag)
        else: self.num_of_channels = 2

        self.layer = nn.Sequential(
            nn.Conv2d(in_channels=self.num_of_channels, out_channels=48, kernel_size=(3, 3), padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),

            # nn.AvgPool2d(stride=2, kernel_size=(2, 2)),

            nn.Conv2d(in_channels=48, out_channels=64, kernel_size=(1, 1), padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # nn.AvgPool2d(stride=2, kernel_size=(2, 2)),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # nn.AvgPool2d(stride=2, kernel_size=(2, 2)),

            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3, 3), padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            # nn.AvgPool2d(stride=2, kernel_size=(2, 2))
        )

        self.layer.apply(self._init_weight)

        # self.fc_layer = nn.Sequential(
        #     nn.Linear(64 * 3 * 3, 128),
        #     nn.BatchNorm1d(128),
        #     nn.ReLU(),
        #
        #     nn.Dropout(0.4),
        #
        #     nn.Linear(128, 1),
        # )

    def forward(self, x):
        out = self.layer(x)
        out = torch.nn.Flatten()(out)
        out = self.fc_layer(out)
        return out

    def _init_weight(self, layer, init_type="Xavier"):
        if isinstance(layer, nn.Conv2d):
            if init_type == "Xavier":
                torch.nn.init.xavier_uniform_(layer.weight)
            elif init_type == "He":
                torch.nn.init.kaiming_uniform_(layer.weight)
##########################################################################################

In [4]:
# Implement DQN Algorithm
for m in range(1, args.max_iteration):
    # Collect h experience (s, a, r, s') with current policy and save at replay memory
    # Read and save Pressure, Oil saturation map from simulation results
    for b in range(1, args.replay_batch):
        # Extract b-th experience data from replay memory
        for u in range(1, args.nn_update_num):
            for i in range(1, args.batch_size):
                # Calculate Target Q-value for all samples in b-th batch experience
                # if well_num == 5 (terminal state):
                #   yi = ri
                # elif well_num < 5 (non-terminal state):
                #   yi = ri + args.discount_factor * max.a'(Q_network(s', a'))
            # Loss calculation: L(theta) = sum(yi - Q_network(s', a')) / args.batch_size
            # Update Q-network parameter: theta = theta - args.learning_rate * grad(L(theta))
    # Decrease tau (temperature parameter of Boltzmann policy)


In [5]:
# Return & Visualize Optimization result
# Final well placement


In [ ]:
# Compare PSO Result & DQN Result
